# Classification

In [17]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
from IPython.display import clear_output
import tensorflow as tf

## Loading data

In [18]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versuicolor', 'Virgnica']

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [19]:
train_y = train.pop('Species')
test_y = test.pop('Species')

## Building the model

In [20]:
def input_fn(features, labels, training=True, batch_size=256):
    
    data_set = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        data_set = data_set.shuffle(1000).repeat()

    return data_set.batch(batch_size)

my_feeature_columns = []
for key in train.keys():
    my_feeature_columns.append(tf.feature_column.numeric_column(key=key))


classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feeature_columns, 
    hidden_units=[30, 10],
    n_classes=3,
    model_dir='./model_classifier')

clear_output()

## Training the model

In [21]:
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000)
clear_output()

In [22]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
clear_output()
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.967



## Prediciting

In [23]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print('Please enter numeric values as prompted.')
for feature in features:
    valid = False
    while not valid:
        val = input(feature + ": ")
        if val.isdigit : 
            valid = True
            print(feature, val)
    
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    propability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * propability))

Please enter numeric values as prompted.
SepalLength 2
SepalWidth 2
PetalLength 2
PetalWidth 2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_classifier\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virgnica" (65.3%)
